In [45]:
class MovieAPI():
    def __init__(self):
        self.index = []
        self.title_li = []
        self.link_li = []
        self.image_li = []
        self.year_li = []
        self.dir_li = []
        self.actor_li = []
        self.rating_li = []
    
    def get_movie(self, movie_list, dir_list, client_id, client_secret):
        import urllib.request
        import xmltodict
        import pandas as pd
        import time
        
        for i, movie in enumerate(movie_list):
            encText = urllib.parse.quote(movie)
            # url = "https://openapi.naver.com/v1/search/movie.json?query=" + encText # JSON 결과
            url = "https://openapi.naver.com/v1/search/movie.xml?query=" + encText # XML 결과
            time.sleep(0.5)
            request = urllib.request.Request(url)
            request.add_header("X-Naver-Client-Id",client_id)
            request.add_header("X-Naver-Client-Secret",client_secret)
            response = urllib.request.urlopen(request)
            rescode = response.getcode()
            if(rescode==200):
                response_body = response.read()
                dict_data = xmltodict.parse(response_body.decode('utf-8'), xml_attribs=False)
                search_li = dict_data['rss']['channel']['item']
                if search_li:
                    if type(search_li) == list:
                        for j in range(len(search_li)):
                            mv_title = search_li[j]['title'].replace('</b>','').replace('<b>','')
                            mv_rating = search_li[j]['userRating']
                            mv_dir = search_li[j]['director']
                            mv_dir = mv_dir.split('|')[:-1] if mv_dir != None else None
                            mv_actor = search_li[j]['actor']
                            mv_actor = mv_actor.split('|')[:-1] if mv_actor != None else None
                            if mv_title == movie and mv_rating != '0.00' and mv_rating != None and mv_dir[0] == dir_list[i]:
                                self.index.append(i)
                                self.title_li.append(mv_title)
                                self.link_li.append(search_li[j]['link'])
                                self.image_li.append(search_li[j]['image'])
                                self.year_li.append(search_li[j]['pubDate'])
                                self.dir_li.append(mv_dir)
                                self.actor_li.append(mv_actor)
                                self.rating_li.append(mv_rating)
                    elif type(search_li) == dict:
                        mv_title = search_li['title'].replace('</b>','').replace('<b>','')
                        mv_dir = search_li['director']
                        mv_dir = mv_dir.split('|')[:-1] if mv_dir != None else None
                        mv_actor = search_li['actor']
                        mv_actor = mv_actor.split('|')[:-1] if mv_actor != None else None

                        self.index.append(i)
                        self.title_li.append(mv_title)
                        self.link_li.append(search_li['link'])
                        self.image_li.append(search_li['image'])
                        self.year_li.append(search_li['pubDate'])
                        self.dir_li.append(mv_dir)
                        self.actor_li.append(mv_actor)
                        self.rating_li.append(search_li['userRating'])
            else:
                print("Error Code:" + rescode)
            if (i != 0 and i % 10 == 0) or i == (len(movie_list)-1):
                print(f'{i}번째 완료')
            if i % 100 == 9:    # 0~99번마다 savepoint로 저장해두기 
                pd.DataFrame([self.index, self.title_li, self.link_li, self.image_li, self.year_li, self.dir_li, self.actor_li, self.rating_li]).transpose().to_csv('savepoint_df_cp949.csv', encoding='cp949')

        movie_df = pd.DataFrame([self.index, self.title_li, self.link_li, self.image_li, self.year_li, self.dir_li, self.actor_li, self.rating_li]).transpose()
        movie_df.columns = ['순번', '영화명', '네이버URL', '이미지URL', '연도', '감독', '출연', '평점']
        movie_df = movie_df.set_index('순번')

        return movie_df

In [42]:
import pandas as pd

# 리스트 불러오기
movie_df = pd.read_csv('movie_resize.csv', encoding='utf-8') #, index_col=0)
display(movie_df[:6])
movie_list = movie_df['영화명'].to_list()
dir_list = movie_df['감독'].to_list()
print('len =',len(movie_list), movie_list[:10])
print('len =',len(dir_list), dir_list[:10])

,영화명,감독,배급사,개봉일,영화형태,국적,전국스크린수,전국 매출액,전국 관객수,장르,등급,영화구분
0,명량,김한민,(주)씨제이이엔엠,2014-07-30,장편,한국,1587,135748398910,17613682,사극,15세관람가,일반영화
1,극한직업,이병헌,(주)씨제이이엔엠,2019-01-23,장편,한국,1978,139647979516,16264944,코미디,15세관람가,일반영화
2,신과함께-죄와 벌,김용화,롯데쇼핑㈜롯데엔터테인먼트,2017-12-20,장편,한국,1912,115698654137,14410754,판타지,12세관람가,일반영화
3,국제시장,윤제균,(주)씨제이이엔엠,2014-12-17,장편,한국,966,110828014630,14245998,드라마,12세관람가,일반영화
4,어벤져스: 엔드게임,안소니 루소,월트디즈니컴퍼니코리아 유한책임회사,2019-04-24,장편,미국,2835,122182694160,13934592,액션,12세관람가,일반영화
5,겨울왕국 2,제니퍼 리,월트디즈니컴퍼니코리아 유한책임회사,2019-11-21,장편,미국,2648,114810421450,13747792,애니메이션,전체관람가,일반영화


len = 4145 ['명량', '극한직업', '신과함께-죄와 벌', '국제시장', '어벤져스: 엔드게임', '겨울왕국 2', '아바타', '베테랑', '도둑들', '7번방의 선물']
len = 4145 ['김한민', '이병헌', '김용화', '윤제균', '안소니 루소', '제니퍼 리', '제임스 카메론', '류승완', '최동훈', '이환경']


In [46]:
client_id = "viO54DlhW87ESywUDwWb"
client_secret = "7ZHAscDgKs"

ma = MovieAPI()
naver_df = ma.get_movie(movie_list=movie_list[:2], dir_list=dir_list, client_id=client_id, client_secret=client_secret)

display(naver_df)
# naver_df.to_csv('naver_api_cp949.csv', encoding='cp949')


1번째 완료


c:\Anaconda3\lib\site-packages\pandas\core\indexes\base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


,영화명,네이버URL,이미지URL,연도,감독,출연,평점
순번,,,,,,,
0,명량,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/09...,2014,[김한민],"[최민식, 류승룡, 조진웅]",8.44
1,극한직업,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/16...,2018,[이병헌],"[류승룡, 이하늬, 진선규, 이동휘, 공명]",8.51


### ▼ 아래는 함수 확인용

In [57]:
import urllib.request
import xmltodict
import pandas as pd

index = []
title_li = []
link_li = []
image_li = []
dir_li = []
actor_li = []
rating_li = []

i = 0
movie = '명량'
encText = urllib.parse.quote(movie)
# url = "https://openapi.naver.com/v1/search/movie.json?query=" + encText # JSON 결과
url = "https://openapi.naver.com/v1/search/movie.xml?query=" + encText # XML 결과
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    dict_data = xmltodict.parse(response_body.decode('utf-8'), xml_attribs=False)
    search_li = dict_data['rss']['channel']['item']
    if type(search_li) == list:
        for j in range(len(search_li)):
            mv_title = search_li[j]['title'].replace('</b>','').replace('<b>','')
            mv_rating = search_li[j]['userRating']
            if mv_title == movie and mv_rating != '0.00':
                index.append(i)
                title_li.append(mv_title)
                link_li.append(search_li[j]['link'])
                image_li.append(search_li[j]['image'])
                dir_li.append(search_li[j]['director'].split('|'))
                actor_li.append(search_li[j]['actor'].split('|'))
                rating_li.append(mv_rating)
    elif type(search_li) == dict:
        mv_title = search_li['title'].replace('</b>','').replace('<b>','')
        index.append(i)
        title_li.append(mv_title)
        link_li.append(search_li['link'])
        image_li.append(search_li['image'])
        dir_li.append(search_li['director'].split('|'))
        actor_li.append(search_li['actor'].split('|'))
        rating_li.append(search_li['userRating'])
else:
    print("Error Code:" + rescode)

naver_df = pd.DataFrame([index, title_li, link_li, image_li, dir_li, actor_li, rating_li]).transpose()
naver_df.columns = ['순번', '영화명', '네이버URL', '이미지URL', '감독', '출연', '평점']

display(naver_df)

,순번,영화명,네이버URL,이미지URL,감독,출연,평점
0,0,겨울왕국,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/10...,"[크리스 벅, 제니퍼 리, ]","[크리스틴 벨, 이디나 멘젤, ]",9.13


In [35]:
import urllib.request
import xmltodict
import pandas as pd

movie = '명량'
encText = urllib.parse.quote(movie)
# url = "https://openapi.naver.com/v1/search/movie.json?query=" + encText # JSON 결과
url = "https://openapi.naver.com/v1/search/movie.xml?query=" + encText # XML 결과
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    dict_data = xmltodict.parse(response_body.decode('utf-8'), xml_attribs=False)
    search_li = dict_data['rss']['channel']['item']

    if type(search_li) == list:
        for j in range(len(search_li)):
            mv_title = search_li[j]['title'].replace('</b>','').replace('<b>','')
            mv_rating = search_li[j]['userRating']
            mv_dir = search_li[j]['director']
            mv_actor = search_li[j]['actor']
            print(search_li[j])
            # if mv_title == movie and mv_rating != '0.00' and mv_dir == True and mv_actor == True:
            #     print(search_li[j])
            print(mv_title == movie, mv_rating != '0.00', mv_dir != None, mv_actor != None)
            print(mv_actor)

{'title': '<b>명량</b>: 회오리 바다를 향하여', 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=133253', 'image': 'https://ssl.pstatic.net/imgmovie/mdi/mit110/1332/133253_P07_142413.jpg', 'subtitle': None, 'pubDate': '2015', 'director': '정세교|김한민|', 'actor': '김한민|오타니 료헤이|이해영|장준녕|권율|', 'userRating': '5.70'}
False True True True
김한민|오타니 료헤이|이해영|장준녕|권율|
{'title': '<b>명량</b>', 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=93756', 'image': 'https://ssl.pstatic.net/imgmovie/mdi/mit110/0937/93756_P93_115513.jpg', 'subtitle': 'ROARING CURRENTS', 'pubDate': '2014', 'director': '김한민|', 'actor': '최민식|류승룡|조진웅|', 'userRating': '8.44'}
True True True True
최민식|류승룡|조진웅|
